In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator as data_augment

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade_left = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_lefteye_2splits.xml')



In [ ]:
# Function to detect faces and eyes in a frame
def detect_face_and_eyes(frame):
        
    normalized_frame = cv2.normalize(frame, None, 0, 255, cv2.NORM_MINMAX)
    frame_u8 = np.uint8(normalized_frame)
    
    gray = cv2.cvtColor(frame_u8, cv2.COLOR_BGR2GRAY)

    # cv2.imshow('Grayscale Image', gray)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    # print(gray.shape)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3)

    #print(faces)

    left_eye_regions = []
    #right_eye_regions = []

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x-10:x+w+10]
        eyes_left = eye_cascade_left.detectMultiScale(roi_gray)
        #eyes_right = eye_cascade_right.detectMultiScale(roi_gray)
            
         # Use the first detected eye for each side
        if len(eyes_left) > 0:
            left_eye = frame[y:y+h, x+eyes_left[0][0]:x+eyes_left[0][0]+eyes_left[0][2]]
            left_eye_regions.append(left_eye
        else:
            left_eye_regions.append(None)


    return left_eye_regions, #right_eye_regions

In [ ]:
# Function to process frames and labels with face detection
def process_frames_with_eyes(frames, target_size=(224, 224)):
    left_eye_frames = []
    
    for frame in frames:
    
        # Detect face and eyes
        left_eye_regions, right_eye_regions = detect_face_and_eyes(frame)

        for left_eye, right_eye in zip(left_eye_regions, right_eye_regions):
            # Resize left eye region

            if (left_eye is None):
                left_eye_frames.append(None)
                continue
                
            if (right_eye is None):
                
                left_eye_frames.append(None)
                continue

            left_eye = cv2.resize(left_eye, target_size)

            # Resize right eye region
            right_eye = cv2.resize(right_eye, target_size)

            # Normalize pixel values
            left_eye = left_eye / 255.0
            right_eye = right_eye / 255.0

            left_eye_frames.append(left_eye)
            
        
        

    return left_eye_frames


In [ ]:
# Process frames with face detection
left_eye_frames, right_eye_frames = process_frames_with_eyes(features)

In [ ]:
# Assume X_train_left, X_train_right, and labels are lists or arrays containing features and labels
# detected using detect_face_and_eyes function

left_eye_filtered = []
labels_filtered = []

for frame_index in range(len(left_eye_frames)):
    left_eye_features = left_eye_frames[frame_index]
    right_eye_features = right_eye_frames[frame_index]
    label = labels[frame_index]

    # Check if either left or right eye features are not None
    if left_eye_features is not None and right_eye_features is not None:
            left_eye_filtered.append(left_eye_features)
            
            labels_filtered.append(label)


print(len(left_eye_filtered), len(labels_filtered))


In [ ]:
# Assuming left_eye_frames and right_eye_frames are lists of left and right eye frames
left_eye_frames = np.array(left_eye_filtered)
labels_filtered = np.array(labels_filtered)

# Convert labels to numeric format
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels_filtered)
labels_encoded = np.array(labels_encoded)

In [ ]:
np.savenpz('eye_data.npz', left_eye_frames = left_eye_frames, right_eye_frames  = right_eye_frames, labels = labels_encoded)

In [2]:
#data augmetation 
data_generate_training = data_augment (rescale=1./255, 
                              shear_range = 0.2,
                              zoom_range = 0.2,
                              fill_mode = "nearest",
                              horizontal_flip = True,
                              width_shift_range = 0.2,
                              height_shift_range = 0.2,
                              validation_split = 0.15)

data_generate_test = data_augment(rescale = 1./255)


In [ ]:
#data preprocessing and augmentation
traind = data_generate_training.flow_from_directory("Training Data",
                                          target_size = (224, 224),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "training")

testd = data_generate_training.flow_from_directory("Training Data"/input/driver-drowsiness-dataset-ddd/Driver Drowsiness Dataset4(DDD4",
                                          target_size = (227, 227),
                                          seed = 123,
                                          batch_size = 32,
                                          subset = "validation")